#### Ancilla Basis Encoding (ABE)

In [ ]:
from qiskit import Aer, execute
import numpy as np

In [ ]:
from qiskit.circuit import ParameterVector

In [ ]:
from qiskit import QuantumCircuit
from math import log2, ceil

In [ ]:
def get_circuit(nq, n_layers):
  theta = ParameterVector('θ', length=nq*n_layers)
  qc = QuantumCircuit(nq)
  qc.h(range(nq))

  for layer_i in range(n_layers):
    # if layer_i%2:
    for qubit_i in range(nq - 1):
      qc.cx(qubit_i, (qubit_i + 1)%nq)
    for qubit_i in range(nq):
      qc.ry(theta[(nq*layer_i)+qubit_i], qubit_i)
  return qc

In [ ]:
def get_projectors(probabilities, n_c):
  P = [0] * n_c
  P1 = [0] * n_c
  for k, v in probabilities.items():
    index = int(k[1:], 2)
    P[index] += v
    if k[0] == '1':
      P1[index] += v
  return P, P1

def objective_value(x: np.ndarray, w: np.ndarray) -> float:
    cost = 0
    for i in range(len(x)):
        for j in range(len(x)):
            cost = cost + w[i,j]*x[i]*(1-x[j])
    return cost

def evaluate_cost(params, circuit, qubo_matrix):
  global ultimate_valid_probabilities
  global penultimate_valid_probabilities
  global optimization_iteration_count
  global n_shots
  optimization_iteration_count += 1
  # Create a copy of the circuit to avoid modifying the original
  working_circuit = circuit.copy()
  # Apply measurements
  working_circuit.measure_all()
  # Bind the parameters to the circuit
  bound_circuit = working_circuit.assign_parameters(params)
  # Run the circuit on a simulator
  simulator = Aer.get_backend('qasm_simulator')
  job = execute(bound_circuit, simulator, shots=n_shots)
  result = job.result()
  counts = result.get_counts()
  # Compute the probabilities of the states
  probabilities = {state: counts[state] / n_shots for state in counts}
  n_c = 2**(len(list(probabilities.keys())[0])-1)
  P, P1 = get_projectors(probabilities, n_c)
  if 0 in P:
    return 100000
  else:
    # Update the last valid probabilities
    penultimate_valid_probabilities = ultimate_valid_probabilities.copy()
    ultimate_valid_probabilities = probabilities.copy()
  cost = 0
  for i in range(len(qubo_matrix)):
    for j in range(len(qubo_matrix)):
      if i==j:
        cost += qubo_matrix[i][j] * P1[i]/P[i]
      else:
        cost += (qubo_matrix[i][j]*P1[i]*P1[j])/(P[i]*P[j])
  # print('min_cut_cost :', min_cut_cost)
  print(f'@ Iteration {optimization_iteration_count} Cost :',cost)
  return cost

def get_final_measurement_binary_string(circuit, params):
  working_circuit = circuit.copy()
  working_circuit.measure_all()
  bound_circuit = working_circuit.assign_parameters(params)
  simulator = Aer.get_backend('qasm_simulator')
  job = execute(bound_circuit, simulator, shots=1024)
  result = job.result()
  counts = result.get_counts()
  probabilities = {state: counts[state] / 1024 for state in counts}
  return probabilities

def decode_probabilities(probabilities):
  binary_solution = []
  n_r = len(list(probabilities.keys())[0])-1
  n_c = 2**(n_r)
  for i in range(n_c):
    if '0' + format(i, 'b').zfill(n_r) in probabilities and '1' + format(i, 'b').zfill(n_r) in probabilities:
      if probabilities['0' + format(i, 'b').zfill(n_r)] > probabilities['1' + format(i, 'b').zfill(n_r)]:
        binary_solution.append(0)
      else:
        binary_solution.append(1)
    elif '0' + format(i, 'b').zfill(n_r) in probabilities:
      binary_solution.append(0)
    elif '1' + format(i, 'b').zfill(n_r) in probabilities:
      binary_solution.append(1)
  return binary_solution

def decode_binary_string(x, height, width):
  mask = np.zeros([height, width])
  for index,segment in enumerate(x):
    mask[index//width,index%width]=segment
  return mask

In [ ]:
# x_1 = 'basis state of the ancilla qubit whose register qubits are in the state (00) with high probability' = 1
# x_2 = 'basis state of the ancilla qubit whose register qubits are in the state (01) with high probability' = 1
# x_3 = 'basis state of the ancilla qubit whose register qubits are in the state (10) with high probability' = 0
# x_4 = 'basis state of the ancilla qubit whose register qubits are in the state (11) with high probability' = 0

In [ ]:
def ancilla_basis_encoding(G, initial_params, n_layers = 1, optimizer_method = 'COBYLA'):
  w = nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(-w)
  qp = max_cut.to_quadratic_program()
  linear = qp.objective.linear.coefficients.toarray(order=None, out=None)
  quadratic = qp.objective.quadratic.coefficients.toarray(order=None, out=None)
  linear = {int(idx):-round(value,2) for idx,value in enumerate(linear[0])}
  quadratic = {(int(iy),int(ix)):-quadratic[iy, ix] for iy, ix in np.ndindex(quadratic.shape) if iy<ix and abs(quadratic[iy, ix])!=0}
  nc = len(linear)
  nr = ceil(log2(nc))
  nq = nr + 1
  qc = get_circuit(nq, n_layers)
  global ultimate_valid_probabilities
  global penultimate_valid_probabilities
  global optimization_iteration_count
  ultimate_valid_probabilities = []
  penultimate_valid_probabilities = []
  optimization_iteration_count = 0
  options = {}
  optimization_result = minimize(
      fun=evaluate_cost,
      x0=initial_params,
      args=(qc, w),
      method=optimizer_method,
      bounds=tuple([(-np.pi, np.pi) for _ in range(len(initial_params))]),
      options=options)
  if not len(ultimate_valid_probabilities):
    return False, [0]*nc, 0, 0, [0]*initial_params
  binary_string_solution = decode_probabilities(ultimate_valid_probabilities)
  minimization_cost = optimization_result.fun
  optimal_params = optimization_result.x
  cut_cost = 0
  cut_cost = objective_value(np.array(list(map(int,binary_string_solution))), w)
  return optimization_result.success, binary_string_solution, minimization_cost, cut_cost, optimal_params


In [ ]:
from scipy.optimize import differential_evolution

In [ ]:
def ancilla_basis_encoding_de(G, initial_params, n_layers = 1):
  w = nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(-w)
  qp = max_cut.to_quadratic_program()
  linear = qp.objective.linear.coefficients.toarray(order=None, out=None)
  quadratic = qp.objective.quadratic.coefficients.toarray(order=None, out=None)
  linear = {int(idx):-round(value,2) for idx,value in enumerate(linear[0])}
  quadratic = {(int(iy),int(ix)):-quadratic[iy, ix] for iy, ix in np.ndindex(quadratic.shape) if iy<ix and abs(quadratic[iy, ix])!=0}
  nc = len(linear)
  nr = ceil(log2(nc))
  nq = nr + 1
  qc = get_circuit(nq, n_layers)
  global ultimate_valid_probabilities
  global penultimate_valid_probabilities
  global optimization_iteration_count
  ultimate_valid_probabilities = []
  penultimate_valid_probabilities = []
  optimization_iteration_count = 0
  
  optimization_result = differential_evolution(
      func=evaluate_cost,
      args = (qc, w),
      bounds=tuple([(-np.pi, np.pi) for _ in range(len(initial_params))])
      )
  if not len(ultimate_valid_probabilities):
    return False, [0]*nc, 0, 0, [0]*initial_params
  binary_string_solution = decode_probabilities(ultimate_valid_probabilities)
  minimization_cost = optimization_result.fun
  optimal_params = optimization_result.x
  cut_cost = 0
  cut_cost = objective_value(np.array(list(map(int,binary_string_solution))), w)
  return optimization_result.success, binary_string_solution, minimization_cost, cut_cost, optimal_params


#### Experiments for Minimal Encoding

In [ ]:
n_shots = 65536

ultimate_valid_probabilities = []
penultimate_valid_probabilities = []

optimization_iteration_count = 0


In [ ]:
base_path = './'

start_layers = 1
max_layers = 5
initial_params_seed = 123
scipy_optimizer_methods = ["COBYLA", "Powell", "CG", "BFGS", "L-BFGS-B", "SLSQP"]


heights = [2,4]

seeds = [111,222,333,444,555]
seeds

In [ ]:
for seed in seeds:
  report_filename = base_path + 'AncillaBasisStateEncoding_' +  str(seed) + '_' + str(n_shots) + '.txt'
  for height in heights:
    width = height
    print(f'height: {height}, width: {width}, n: {height*width}')
    # G, image = generate_binary_problem_instance(height, width)
    np.random.seed(seed=seed)
    G, image = generate_problem_instance(height, width)
    print("Image Generated: ",image)
    # plt.imshow(image, cmap=plt.cm.gray)
    # plt.show()

    for scipy_optimizer_method in scipy_optimizer_methods:
      print("Maximum number of layers :", max_layers)
      for n_layers in range(start_layers, max_layers+1,1):
        nc = len(G.nodes())
        nr = ceil(log2(nc))
        nq = nr + 1
        initial_params = np.random.uniform(low=-np.pi, high=np.pi, size=nq*n_layers)
        print(f"Executing QC with {n_layers} layers and {scipy_optimizer_method} optimizer for {height}*{height} image.")
        # try:
        start_time = time.time()
        success_flag, minimal_encoding_solution, minimal_encoding_value, minimal_encoding_cut_value, optimal_params = ancilla_basis_encoding_de(G,
                                                                                                                      initial_params,
                                                                                                                      n_layers = n_layers,
                                                                                                                      optimizer_method = scipy_optimizer_method)
        minimal_encoding_tte = (time.time() - start_time)
        print("New NISQ done for",scipy_optimizer_method,"optimizer with a success status :", success_flag)
        print(f"Appending the results of {height}*{height} image using QC with {n_layers} layers and {scipy_optimizer_method} optimizer.")
        row = []
        row.append(int(G.number_of_nodes()))
        row.append(int(height))
        row.append(int(width))
        row.append(success_flag)
        row.append(''.join(map(str, map(int, (minimal_encoding_solution)))))
        row.append(np.round(minimal_encoding_tte,6))
        row.append(n_layers)
        row.append(np.round(minimal_encoding_cut_value,4))
        row.append(np.round(minimal_encoding_value,4))
        row.append(optimization_iteration_count)
        row.append(scipy_optimizer_method)
        # row.append(''.join(map(str, map(float, (optimal_params)))))
        report_file_obj = open(os.path.join(report_filename),'a+')
        report_file_obj.write('__'.join(map(str,row))+'\n')
        report_file_obj.close()
    print('\n')

In [ ]:
def truncate_long_string(s, max_length=20):
    return s if len(s) <= max_length else s[:max_length] + "..."

def style_df(df):
    # Styles for hover and headers
    cell_hover = {'selector': 'tr:hover', 'props': [('background-color', '#707070')]}  # Dark pale color for hover
    headers = {'selector': 'th', 'props': 'background-color: #1D1D1D; color: white;'}

    # Column separator style
    column_separator = {'selector': 'td, th', 'props': 'border-right: 1px solid white;'}

    # Apply the styles
    styled_df = df.style.set_table_styles([cell_hover, headers, column_separator])

    # Style to hide the border for the last column to prevent a double line at the end of the table
    hide_last_border = {'selector': 'td:last-child, th:last-child', 'props': 'border-right: none;'}

    # Combine all styles
    styles = [cell_hover, headers, column_separator, hide_last_border]
    styled_df = styled_df.set_table_styles(styles, overwrite=False)

    return styled_df

In [ ]:
seed = 111

In [ ]:
base_path = './'
report_filename = base_path + 'minimalEncoding_' +  str(seed) + '_' + str(65536) + '.txt'
report_file_obj = open(report_filename,'r')
table_contents = [line.replace('\n','').split('__') for line in report_file_obj.readlines()]

base_cols = ['No. of Pixels', 'Height', 'Width']
sub_cols = ['','','']

extra_sub_cols = ['Optimization Success','Result', 'TTE', 'Layers', 'Min-Cut Cost', 'Optimization Cost', 'Iterations', 'Optimizer Method']
base_cols = base_cols+['Minimal Encoding Solver']*len(extra_sub_cols)
sub_cols=sub_cols+extra_sub_cols

column_arrays = [base_cols, sub_cols]

df = pd.DataFrame(table_contents, columns=pd.MultiIndex.from_arrays(column_arrays))

for col in df.columns:
  if len(col) == 2 and col[1] == 'Result':
    df[col] = df[col].apply(lambda x: truncate_long_string(x))

styled_df = style_df(df)#.apply(highlight_row, axis=1)




styled_df